# openai tools

- https://platform.openai.com/docs/guides/function-calling
- https://cookbook.openai.com/examples/using_tool_required_for_customer_service

In [38]:
from openai import AzureOpenAI
import json

with open("openai_api.json") as f:
    creds = json.load(f)

client = AzureOpenAI(
    # https://learn.microsoft.com/azure/ai-services/openai/reference#rest-api-versioning
    api_version="2024-06-01",
    # https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=creds["api_base"],
    api_key=creds["api_key"],
)

In [18]:
tools = [
    {
      "type": "function",
      "function": {
        "name": "get_order_details",
        "description": "Use this to get order details like date of purchase, status, cost, etc.",
        "parameters": {
          "type": "object",
          "properties": {
            "product_name": {
              "type": "string",
              "description": "Order's Product name."
            }
          },
          "required": ["product_name"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "list_orders",
        "description": "Use this to get list of all orders with their product name, date of purchase, status.",
        "result": {
            "type": "object",
            "properties": {
                "product_name": {
                  "type": "string",
                  "description": "Product name."
                },
                "purchase_date": {
                  "type": "string",
                  "description": "Purchase date."
                },
                "status": {
                  "type": "string",
                  "description": "Current status."
                },
            },
        }
      }
    }
]

In [19]:
messages = [
    {
        "role": "system",
        "content": "You are an AI assistant for an ecommerce app helping customers with their queries about their orders.",
    },
    {
        "role": "user",
        "content": "where's my order for nike jordan shoes?"
    }
]

In [48]:
completion = client.chat.completions.create(
    model="4o-mini",
    temperature=0,
    messages=messages,
    tools=tools,
    parallel_tool_calls=False,
    tool_choice="required",  # https://platform.openai.com/docs/guides/function-calling/configuring-function-calling-behavior-using-the-tool_choice-parameter
)

In [49]:
print("Response:", completion.to_json())

Response: {
  "id": "chatcmpl-AJd78ujp8gu5jgsZrYcaDBM2Zhj78",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_B4x4WeimboMHvmmtkvIdZSHt",
            "function": {
              "arguments": "{\"product_name\":\"nike jordan shoes\"}",
              "name": "get_order_details"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1729241950,
  "model": "gpt-4o-mini",
  "object": "chat.completion",
  "system_fingerprint": "fp_878413d04d",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 118,
    "total_tokens": 134
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "jailbreak": 

In [55]:
choice = completion.choices[0]

In [51]:
messages2 = messages + [
    choice.message.to_dict(),
    {
        "role": "tool",
        "content": json.dumps({
            "product_name": "Nike Jordan Shoes",
            "purchase_date": "18 Oct 2024",
            "status": "Out for delivery", 
        }),
        "tool_call_id": choice.message.tool_calls[0].id
    },
]

In [52]:
messages2

[{'role': 'system',
  'content': 'You are an AI assistant for an ecommerce app helping customers with their queries about their orders.'},
 {'role': 'user', 'content': "where's my order for nike jordan shoes?"},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_B4x4WeimboMHvmmtkvIdZSHt',
    'function': {'arguments': '{"product_name":"nike jordan shoes"}',
     'name': 'get_order_details'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '{"product_name": "Nike Jordan Shoes", "purchase_date": "18 Oct 2024", "status": "Out for delivery"}',
  'tool_call_id': 'call_B4x4WeimboMHvmmtkvIdZSHt'}]

In [53]:
completion2 = client.chat.completions.create(
    model="4o-mini",
    temperature=0,
    messages=messages2,
    tools=tools,
)

In [54]:
print("Response:", completion2.to_json())

Response: {
  "id": "chatcmpl-AJd80WYDG26lWDVC6BQnyuwMsoG20",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Your order for the Nike Jordan Shoes was purchased on October 18, 2024, and is currently \"Out for delivery.\"",
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1729242004,
  "model": "gpt-4o-mini",
  "object": "chat.completion",
  "system_fingerprint": "fp_878413d04d",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 171,
    "total_tokens": 198
  },
  "prompt_filter_results